In [3]:
import numpy as np

In [8]:
a = np.arange(3)

In [9]:
b = np.random.rand(5,10)
b

array([[0.75841008, 0.54868016, 0.83428867, 0.27385056, 0.91753644,
        0.08942583, 0.21275664, 0.38712002, 0.44738196, 0.7896003 ],
       [0.67633079, 0.87801387, 0.11897228, 0.82442385, 0.56319763,
        0.34887457, 0.58571101, 0.7563428 , 0.23349702, 0.89903099],
       [0.88747378, 0.86488877, 0.61475251, 0.58124001, 0.10442912,
        0.25763182, 0.06630175, 0.0089038 , 0.03738588, 0.42839326],
       [0.03539434, 0.45719071, 0.09415967, 0.57879717, 0.25503488,
        0.08870671, 0.96658742, 0.55322771, 0.06440132, 0.16539705],
       [0.36089616, 0.69203234, 0.37574307, 0.85412597, 0.93688369,
        0.81450029, 0.93680354, 0.61661656, 0.48130541, 0.56918184]])

In [11]:
b[[1,2,3]]

array([[0.67633079, 0.87801387, 0.11897228, 0.82442385, 0.56319763,
        0.34887457, 0.58571101, 0.7563428 , 0.23349702, 0.89903099],
       [0.88747378, 0.86488877, 0.61475251, 0.58124001, 0.10442912,
        0.25763182, 0.06630175, 0.0089038 , 0.03738588, 0.42839326],
       [0.03539434, 0.45719071, 0.09415967, 0.57879717, 0.25503488,
        0.08870671, 0.96658742, 0.55322771, 0.06440132, 0.16539705]])

In [12]:
a = np.random.rand(3,4,5)
b = np.random.rand(5,4)

In [14]:
c = b @ a

In [15]:
c.shape

(3, 5, 5)